<a href="https://colab.research.google.com/github/RAFS20/Data-Science/blob/main/Funciones_de_Estad%C3%ADstica_Descriptiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install pandas numpy matplotlib seaborn scipy scikit-learn statsmodels python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.1 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np

# Fijar la semilla para reproducibilidad
np.random.seed(42)
# Número de muestras
num_samples = 100
# Generar variables numéricas
edad = np.random.randint(18, 65, size=num_samples)
ingresos = np.random.normal(50000, 15000, size=num_samples).round(2)
puntaje_credito = np.random.randint(300, 850, size=num_samples)
# Generar variables categóricas
genero = np.random.choice(['Masculino', 'Femenino', 'No especificado'], size=num_samples)
estado_civil = np.random.choice(['Soltero', 'Casado', 'Divorciado', 'Viudo'], size=num_samples)
educacion = np.random.choice(['Secundaria', 'Universidad', 'Postgrado', 'Doctorado'], size=num_samples)
# Crear un DataFrame
data = {
    'Edad': edad,
    'Ingresos': ingresos,
    'PuntajeCredito': puntaje_credito,
    'Genero': genero,
    'EstadoCivil': estado_civil,
    'Educacion': educacion
}
df = pd.DataFrame(data)
# Mostrar las primeras filas del DataFrame
print(df.head())
# Guardar el DataFrame en un archivo CSV
df.to_csv('dataset.csv', index=False)

   Edad  Ingresos  PuntajeCredito           Genero EstadoCivil    Educacion
0    56  59544.58             443         Femenino      Casado  Universidad
1    46  36399.19             645         Femenino      Casado    Doctorado
2    32  57140.64             301  No especificado     Soltero   Secundaria
3    60  69554.92             603  No especificado       Viudo    Doctorado
4    25  53173.81             553  No especificado      Casado   Secundaria


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import os

# Función para cargar el dataset
def cargar_dataset(filepath):
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.xlsx'):
        return pd.read_excel(filepath)
    else:
        raise ValueError("Formato de archivo no soportado")

# Función para estadística descriptiva
def estadistica_descriptiva(df):
    resumen = df.describe(include='all').transpose()
    print("Resumen Estadístico:\n", resumen)
    return resumen

# Función para visualización de datos
def visualizar_datos(df, doc):
    if not os.path.exists('output'):
        os.makedirs('output')
    for col in df.select_dtypes(include=[np.number]).columns:
        plt.figure(figsize=(10, 4))
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(f'Distribución de {col}')
        hist_path = f'output/hist_{col}.png'
        plt.savefig(hist_path)
        plt.close()
        doc.add_heading(f'Distribución de {col}', level=2)
        doc.add_picture(hist_path, width=Inches(6))

        plt.figure(figsize=(10, 4))
        sns.boxplot(x=df[col].dropna())
        plt.title(f'Boxplot de {col}')
        box_path = f'output/box_{col}.png'
        plt.savefig(box_path)
        plt.close()
        doc.add_heading(f'Boxplot de {col}', level=2)
        doc.add_picture(box_path, width=Inches(6))

    for col in df.select_dtypes(include=[object]).columns:
        plt.figure(figsize=(10, 4))
        sns.countplot(x=df[col].dropna())
        plt.title(f'Conteo de {col}')
        count_path = f'output/count_{col}.png'
        plt.savefig(count_path)
        plt.close()
        doc.add_heading(f'Conteo de {col}', level=2)
        doc.add_picture(count_path, width=Inches(6))

# Función para cruce de variables
def cruce_variables(df, doc):
    for col1 in df.select_dtypes(include=[object]).columns:
        for col2 in df.select_dtypes(include=[object]).columns:
            if col1 != col2:
                tabla_contingencia = pd.crosstab(df[col1], df[col2])
                plt.figure(figsize=(10, 6))
                sns.heatmap(tabla_contingencia, annot=True, fmt="d", cmap="YlGnBu")
                plt.title(f'Cruz de {col1} y {col2}')
                cross_path = f'output/cross_{col1}_{col2}.png'
                plt.savefig(cross_path)
                plt.close()
                doc.add_heading(f'Cruz de {col1} y {col2}', level=2)
                doc.add_picture(cross_path, width=Inches(6))

# Función para análisis inferencial
def analisis_inferencial(df, doc):
    # Prueba t entre dos primeras columnas numéricas
    num_cols = df.select_dtypes(include=[np.number]).columns
    if len(num_cols) > 1:
        t_stat, p_val = stats.ttest_ind(df[num_cols[0]].dropna(), df[num_cols[1]].dropna())
        doc.add_heading(f'Prueba t entre {num_cols[0]} y {num_cols[1]}', level=2)
        doc.add_paragraph(f't = {t_stat}, p = {p_val}')

    # ANOVA para más de dos grupos categóricos
    cat_cols = df.select_dtypes(include=[object]).columns
    if len(cat_cols) > 0 and len(num_cols) > 0:
        for col in cat_cols:
            if df[col].nunique() > 1:
                formula = f'{num_cols[0]} ~ C({col})'
                model = smf.ols(formula, data=df).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)
                doc.add_heading(f'ANOVA para {num_cols[0]} según {col}', level=2)
                doc.add_paragraph(anova_table.to_string())

    # Chi-cuadrado para independencia de variables categóricas
    if len(cat_cols) > 1:
        chi2, p, dof, ex = stats.chi2_contingency(pd.crosstab(df[cat_cols[0]], df[cat_cols[1]]))
        doc.add_heading(f'Chi-cuadrado entre {cat_cols[0]} y {cat_cols[1]}', level=2)
        doc.add_paragraph(f'chi2 = {chi2}, p = {p}')

# Función para análisis de correlación
def analisis_correlacion(df, doc):
    # Crear un DataFrame solo con variables numéricas y dummies
    df_numerico = pd.get_dummies(df, drop_first=True)
    corr = df_numerico.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
    plt.title('Matriz de Correlación')
    corr_path = 'output/correlation_matrix.png'
    plt.savefig(corr_path)
    plt.close()
    doc.add_heading('Matriz de Correlación', level=2)
    doc.add_picture(corr_path, width=Inches(6))

# Función para PCA
def analisis_pca(df, doc):
    scaler = StandardScaler()
    num_data = pd.get_dummies(df.select_dtypes(include=[np.number, object]), drop_first=True).dropna(axis=0)
    if num_data.shape[0] > 0 and num_data.shape[1] > 1:
        scaled_data = scaler.fit_transform(num_data)
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(scaled_data)
        plt.figure(figsize=(10, 8))
        plt.scatter(pca_result[:, 0], pca_result[:, 1])
        plt.title('Análisis de Componentes Principales')
        plt.xlabel('Componente Principal 1')
        plt.ylabel('Componente Principal 2')
        pca_path = 'output/pca.png'
        plt.savefig(pca_path)
        plt.close()
        doc.add_heading('Análisis de Componentes Principales', level=2)
        doc.add_picture(pca_path, width=Inches(6))

# Función para detección de outliers
def detectar_outliers(df, doc):
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
        if not outliers.empty:
            doc.add_heading(f'Outliers en {col}', level=2)
            doc.add_paragraph(outliers.to_string())

# Función para generar reporte
def generar_reporte(df, resumen, doc):
    doc.add_heading('Resumen Estadístico', level=1)
    table = doc.add_table(rows=1, cols=len(resumen.columns) + 1)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Variable'
    for i, col_name in enumerate(resumen.columns):
        hdr_cells[i + 1].text = col_name

    for idx, row in resumen.iterrows():
        row_cells = table.add_row().cells
        row_cells[0].text = idx
        for i, value in enumerate(row):
            row_cells[i + 1].text = str(value)

# Función principal
def main(filepath):
    doc = Document()
    doc.add_heading('Reporte de Análisis de Datos', 0)

    df = cargar_dataset(filepath)
    resumen = estadistica_descriptiva(df)
    generar_reporte(df, resumen, doc)
    visualizar_datos(df, doc)
    cruce_variables(df, doc)
    analisis_inferencial(df, doc)
    analisis_correlacion(df, doc)
    analisis_pca(df, doc)
    detectar_outliers(df, doc)

    doc.save('reporte.docx')

# Ejecución
if __name__ == "__main__":
    main('dataset.csv')

Resumen Estadístico:
                 count unique         top freq        mean           std  \
Edad            100.0    NaN         NaN  NaN       40.88      13.99082   
Ingresos        100.0    NaN         NaN  NaN  51027.6435  14965.841917   
PuntajeCredito  100.0    NaN         NaN  NaN      557.28    162.329325   
Genero            100      3    Femenino   40         NaN           NaN   
EstadoCivil       100      4       Viudo   30         NaN           NaN   
Educacion         100      4  Secundaria   34         NaN           NaN   

                     min       25%        50%       75%       max  
Edad                18.0      30.5       41.0     53.25      64.0  
Ingresos        20901.33  38517.79  49870.225  61300.23  94154.95  
PuntajeCredito     301.0     425.0      548.5    697.75     846.0  
Genero               NaN       NaN        NaN       NaN       NaN  
EstadoCivil          NaN       NaN        NaN       NaN       NaN  
Educacion            NaN       NaN        Na